In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

Load anndata

In [2]:
adata = sc.read_h5ad("test.h5ad")

In [3]:
# lognorm should be the default (expected)
adata.X = adata.layers['logcounts']

In [4]:
sc.pp.log1p(adata)

In [5]:
adata.X.data

array([0.89279661, 0.86977312, 0.96258582, ..., 0.67557673, 0.63240171,
       0.93567513])

Parameters

In [6]:
groupby='label'

In [7]:
de_method='wilcoxon'

In [8]:
resource_name = 'consensus'

In [9]:
complex_policy = 'min'

In [10]:
key_cols = ['source', 'target', 'ligand_complex', 'receptor_complex']

In [11]:
verbose = False

In [12]:
# only relevant if perms are done
n_perms = 1000
seed = 69

In [13]:
# I get this from the score object
_complex_cols = ['ligand_means', 'receptor_means']
# change to full list and move to _var
_add_cols = ['ligand', 'receptor',
             'ligand_means_sums', 'receptor_means_sums',
             'ligand_zscores', 'receptor_zscores',
             'ligand_logfoldchanges', 'receptor_logfoldchanges',
             'mat_mean',
             ]

_key_cols = ['source', 'target', 'ligand_complex', 'receptor_complex']

Run LIANA PIPE

In [14]:
from liana import liana_pipe

In [15]:
lr_res = liana_pipe(adata=adata, 
                    groupby=groupby, 
                    resource_name=resource_name,
                    de_method=de_method,
                    n_perms=n_perms,
                    seed=seed,
                    verbose=verbose,
                    _key_cols=key_cols,
                    resource=None)

/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value
/home/dbdimitrov/anaconda3/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)


In [16]:
lr_res

,ligand,ligand_complex,ligand_logfoldchanges,ligand_means,ligand_zscores,mat_mean,receptor,receptor_complex,receptor_logfoldchanges,receptor_means,receptor_zscores,source,target,ligand_min,receptor_min,ligand_means_sums,receptor_means_sums
0,LGALS9,LGALS9,2.331637,0.220470,0.379478,0.111122,PTPRC,PTPRC,-0.826139,0.479144,-0.349941,B,B,0.220470,0.479144,0.316148,1.953287
1,CD22,CD22,3.585404,0.346348,0.639254,0.111122,PTPRC,PTPRC,-0.826139,0.479144,-0.349941,B,B,0.346348,0.479144,0.414147,1.953287
2,LGALS1,LGALS1,-1.402231,0.350487,-0.442577,0.111122,PTPRC,PTPRC,-0.826139,0.479144,-0.349941,B,B,0.350487,0.479144,1.843355,1.953287
3,LGALS9,LGALS9,2.331637,0.220470,0.379478,0.111122,CD44,CD44,-0.073450,0.393289,-0.022623,B,B,0.220470,0.393289,0.316148,1.213549
4,ADAM10,ADAM10,0.074842,0.078512,0.010363,0.111122,CD44,CD44,-0.073450,0.393289,-0.022623,B,B,0.078512,0.393289,0.227885,1.213549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4576,CALM1,CALM1,0.593629,1.238195,0.477889,0.111122,KCNN4,KCNN4,-25.267597,0.000000,-0.183999,NK,NK,1.238195,0.000000,3.166939,0.079197
4577,PTPN6,PTPN6,-0.589895,0.383890,-0.199303,0.111122,CD300LF,CD300LF,-23.912775,0.000000,-0.105409,NK,NK,0.383890,0.000000,1.450293,0.031339
4578,NUCB2,NUCB2,0.409608,0.093687,0.057988,0.111122,ERAP1,ERAP1,2.764175,0.096286,0.277586,NK,NK,0.093687,0.096286,0.236354,0.125823
4579,SOCS2,SOCS2,1.655334,0.132848,0.230284,0.111122,EPOR,EPOR,-23.893091,0.000000,-0.105409,NK,NK,0.132848,0.000000,0.221081,0.030918


CellPhoneDB

In [17]:
from liana import cellphonedb

In [18]:
%%time
adata = cellphonedb(adata, groupby='label', n_perms=1000)

/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value
/home/dbdimitrov/anaconda3/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)


CPU times: user 4.08 s, sys: 10.3 ms, total: 4.09 s
Wall time: 4.07 s


In [19]:
adata.uns['liana_res']

,ligand,ligand_complex,ligand_means,receptor,receptor_complex,receptor_means,source,target,ligand_min,receptor_min,lr_means,pvals
0,LGALS9,LGALS9,0.220470,PTPRC,PTPRC,0.479144,B,B,0.220470,0.479144,0.349807,0.744
1,CD22,CD22,0.346348,PTPRC,PTPRC,0.479144,B,B,0.346348,0.479144,0.412746,0.300
2,LGALS1,LGALS1,0.350487,PTPRC,PTPRC,0.479144,B,B,0.350487,0.479144,0.414815,1.000
3,LGALS9,LGALS9,0.220470,CD44,CD44,0.393289,B,B,0.220470,0.393289,0.306880,0.102
4,ADAM10,ADAM10,0.078512,CD44,CD44,0.393289,B,B,0.078512,0.393289,0.235900,0.514
...,...,...,...,...,...,...,...,...,...,...,...,...
4576,CALM1,CALM1,1.238195,KCNN4,KCNN4,0.000000,NK,NK,1.238195,0.000000,1.000000,1.000
4577,PTPN6,PTPN6,0.383890,CD300LF,CD300LF,0.000000,NK,NK,0.383890,0.000000,1.000000,1.000
4578,NUCB2,NUCB2,0.093687,ERAP1,ERAP1,0.096286,NK,NK,0.093687,0.096286,0.094987,0.069
4579,SOCS2,SOCS2,0.132848,EPOR,EPOR,0.000000,NK,NK,0.132848,0.000000,1.000000,1.000


NATMI

In [ ]:
from liana import natmi

In [ ]:
adata = natmi(adata, groupby='label')

In [ ]:
adata.uns['liana_res']

SCA

In [ ]:
from liana import singlecellsignalr as sca

In [ ]:
adata = sca(adata, groupby='label')

In [ ]:
adata.uns['liana_res'].sort_values('lr_means', ascending=True)

Connectome

In [ ]:
from liana import connectome

In [ ]:
adata = connectome(adata, groupby='label')

In [ ]:
adata.uns['liana_res'].sort_values('scaled_weight', ascending=False)

logFC

In [ ]:
from liana import logfc

In [ ]:
adata = logfc(adata, groupby='label')

In [ ]:
adata.uns['liana_res'].sort_values('lr_logfc', ascending=False)

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='label', method='t-test')

In [ ]:
adata.uns['rank_genes_groups']

try w/ real data

In [ ]:
adata = sc.datasets.pbmc3k_processed()
labels = adata.obs.louvain
cells = adata.obs_names

In [ ]:
adata = sc.datasets.pbmc3k()

In [ ]:
adata = adata[[x in cells for x in adata.obs_names]]

In [ ]:
adata.obs['label'] = labels

In [ ]:
sc.pp.log1p(adata)

In [ ]:
%%time
adata = cellphonedb(adata, groupby='label', n_perms=1000)

Calculate means_sums for NATMI

In [ ]:
def _sum_means(lr_res, what, on):
    return lr_res.join(lr_res.groupby(on)[what].sum(), on=on, rsuffix='_sums')

In [ ]:
lr_res = _sum_means(lr_res, what='ligand_means', on=['ligand_complex', 'receptor_complex', 'target'])
lr_res = _sum_means(lr_res, what='receptor_means', on=['ligand_complex', 'receptor_complex', 'source'])

In [ ]:
lr_res

NATMI fun

Both magnitude and specificity! Returned to unpack!!!

In [ ]:
lr_res[['expr_prod','spec_weight']] = lr_res.apply(_natmi_score, axis=1, result_type="expand")

In [ ]:
lr_res[lr_res['edge_specificity']==1].sort_values('ligand')

#### LIANA PIPE LINE BY LINE

In [ ]:
from liana.utils.pre import check_adata, check_if_covered, format_vars, filter_resource
from liana.resource import select_resource
from liana.resource.select_resource import explode_complexes
from liana.steady.liana_pipe import _get_lr
from liana.utils.reassemble_complexes import reassemble_complexes
from scipy.sparse import csr_matrix
from liana.steady._permutations import get_means_perms

I need to double check if every step makes sense, as I build unit tests!!!

For example, are there any duplicated rows (source, target, ligand_complex, receptor_complex)

Double check again if correct subunit is kept, etc

In [ ]:
adata = check_adata(adata, True)

In [ ]:
# Define idents col name
adata.obs.label = adata.obs[groupby]

Load resource

In [ ]:
resource = select_resource(resource_name='consensus')

In [ ]:
# Decomplexify
resource = explode_complexes(resource)

In [ ]:
# Filter Resource
resource = filter_resource(resource, adata.var_names)

Create entities

In [ ]:
entities = np.union1d(np.unique(resource["ligand"]), np.unique(resource["receptor"]))

Check overlap between resource and adata

In [ ]:
check_if_covered(entities, adata.var_keys)

Get global mean for SCA before filtering

In [ ]:
if 'mat_mean' in _add_cols: # SHOULD BE METHOD NAME!
    adata.uns['mat_mean'] = np.mean(adata.X)

In [ ]:
# Filter to only include the relevant genes
adata = adata[:,np.intersect1d(entities, adata.var.index)]
adata

Get Stats

In [ ]:
lr_res = _get_lr(adata, resource, key_cols + _complex_cols + _add_cols, de_method)

In [ ]:
lr_res

Recomplexify

In [ ]:
# If I want to implement anything else but the min as complex policy, I would need to change this:
# temp = temp[[x=="INHBA_INHBB" for x in temp['ligand_complex']]].sort_values('receptor_complex')
# temp = temp[[x=="ACVR1B_ACVR2A" for x in temp['receptor_complex']]]

In [ ]:
lr_res = reassemble_complexes(lr_res, _key_cols, _complex_cols)

In [ ]:
lr_res

### SCORES

In [ ]:
from liana.steady.Method import Method, MethodMeta
from liana.steady.scores.cellphonedb import _cpdb_score

CELLPHONEDB

In [ ]:
_cellphonedb = MethodMeta(method_name = "cellphonedb",
                          complex_cols = ['ligand_means', 'receptor_means'], 
                          add_cols = ['ligand', 'receptor'],
                          fun = _cpdb_score,
                          magnitude = 'lr_means',
                          specificity = 'pvals',
                          permute = True,
                          reference = 'Efremova et al., 2020')

In [ ]:
cellphonedb = Method(_SCORE=_cellphonedb)

In [ ]:
adata = cellphonedb(adata, groupby, seed=666)

In [ ]:
adata.uns['liana_res']

logFC re-implement

Magnitude as None

In [ ]:
lr_res['logfc'] = lr_res[['ligand_logfoldchanges', 'receptor_logfoldchanges']].mean(1)

In [ ]:
lr_res.sort_values(by='logfc', key=abs, ascending=False)

Re-implement Connectome

Both Specificity and Magnitude

In [ ]:
lr_res['edge_weight'] = lr_res[['ligand_zscores', 'receptor_zscores']].mean(1)